In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from scipy.fftpack import fftn

def porods_law_over_time(data, q_min=1.0, q_max=10.0):
    """
    Calculate Porod's law over time for a system represented by a 3D numpy array.
    The first dimension of the array should represent time, and the last two dimensions should represent the system state.
    """
    # Define the power law function
    def power_law(q, K, P):
        return K * q**(-P)

    # Initialize an array to hold the Porod's exponents
    porod_exponents = np.zeros(data.shape[0])

    # Loop over time steps
    for i in range(data.shape[0]):
        # Compute the structure factor
        structure_factor = np.abs(fftn(data[i]))**2

        # Compute the magnitude of the wavevector for each point in the structure factor
        q_values = np.sqrt(np.sum(np.array(np.meshgrid(*[np.fft.fftfreq(n) * n for n in data.shape[1:]]))**2, axis=0))

        # Flatten the q_values and structure_factor arrays for curve fitting
        q_values = q_values.flatten()
        structure_factor = structure_factor.flatten()

        # Fit the tail of the structure factor to a power law
        mask = (q_values >= q_min) & (q_values <= q_max)
        popt, pcov = curve_fit(power_law, q_values[mask], structure_factor[mask])

        # The second parameter of the fit is the Porod's exponent
        porod_exponents[i] = popt[1]

    return porod_exponents